## Extração das Perguntas e Respostas do IRPF

- Dados em data/

In [ ]:
from docling.document_converter import DocumentConverter

source = "/home/lucas/Desktop/slm-ir-assistant/data/pdfs/perguntas_respostas_2022_IRPF.pdf"
converter = DocumentConverter()
doc = converter.convert(source).document

# print(doc.export_to_markdown())


[INFO] 2026-02-11 19:29:42,752 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-11 19:29:42,753 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-11 19:29:42,796 [RapidOCR] download_file.py:60: File exists and is valid: /home/lucas/Desktop/slm-ir-assistant/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-11 19:29:42,797 [RapidOCR] main.py:50: Using /home/lucas/Desktop/slm-ir-assistant/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-11 19:29:43,046 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-11 19:29:43,047 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-11 19:29:43,049 [RapidOCR] download_file.py:60: File exists and is valid: /home/lucas/Desktop/slm-ir-assistant/.venv/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-11 19:29:43,050 [RapidOCR] main.py:50: Using /home/lucas

In [14]:
# import re

# text = doc.export_to_markdown()

# re.findall(r'(?<!\d)\d{3}\s+-', text)

In [15]:
# 2. Save that string to a file
with open("perguntas_respostas_2023_IRPF.md", "w", encoding="utf-8") as f:
    f.write(doc.export_to_markdown())

In [21]:
import re
import json

def extract_and_validate_rfb_questions(text):
    # 1. Regex Configuration
    # Anchored to start of line, optional '##', negative lookbehind for digits
    id_pattern = re.compile(r'^\s*(?:##\s*)?(?<!\d)(\d{3})\s+-\s*')
    
    # Matches lines starting with "##" (Topics/Metadata)
    header_pattern = re.compile(r'^##\s+(.*)')
    
    questions = []
    current_topic = "Geral" # Default metadata
    
    # Buffers to hold multi-line text
    current_lines = []
    current_id = None
    current_topic_snapshot = None

    lines = text.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line: continue
        
        # SKIP specific navigation lines if they appear strictly on their own
        if "Retorno ao sumário" in line:
            continue

        # CHECK: Is this a Question? (Priority check)
        id_match = id_pattern.match(line)
        
        if id_match:
            # Save the PREVIOUS question found (if any)
            if current_id:
                questions.append(process_entry(current_id, current_topic_snapshot, current_lines))
            
            # Start a NEW question
            current_id = id_match.group(1)
            current_topic_snapshot = current_topic
            current_lines = []
            
            # Remove the ID part from the line to get the question text started
            content_start = id_match.end()
            current_lines.append(line[content_start:].strip())
            
        # CHECK: Is this a Metadata Header? (Only if it's not a question)
        elif header_pattern.match(line):
            header_text = header_pattern.match(line).group(1).strip()
            # Ignore headers that are likely sub-sections inside an answer
            # We also ignore "Retorno" here just in case
            ignore_keywords = ["Atenção", "Dispensa", "Atividade rural", "Retorno"]
            if not any(keyword in header_text for keyword in ignore_keywords):
                current_topic = header_text
        
        # OTHERWISE: It's part of the current Question/Answer text
        else:
            if current_id:
                current_lines.append(line)

    # Don't forget the very last question!
    if current_id:
        questions.append(process_entry(current_id, current_topic_snapshot, current_lines))

    return questions

def process_entry(q_id, topic, lines):
    # Join lines with newline to preserve structure for regex (important for citations)
    full_text = "\n".join(lines)
    
    # 1. Split Question and Answer at the first '?'
    if '?' in full_text:
        parts = full_text.split('?', 1)
        question_text = parts[0].strip() + "?"
        answer_text = parts[1].strip()
    else:
        question_text = full_text
        answer_text = ""

    # 2. CLEAN FOOTERS (Consulte / Retorno)
    # Remove these lines from the end to expose the legal citation
    # Regex: Look for lines starting with "Consulte" or "Retorno" at the end of the text
    # (?m) allows ^ to match the start of each line
    answer_text = re.sub(r'(?m)^\s*(Consulte|Retorno).*$', '', answer_text).strip()

    # 3. EXTRACT LEGAL SOURCE (Robust Multi-line)
    # Expanded keywords to catch all variations
    legal_keywords = r"(?:Lei|Decreto|Instrução|Regulamento|RIR|Ato|Solução|Parecer|Portaria|Constituição|Emenda|Medida)"
    
    # Regex logic:
    # \(\s*{kw}   -> Starts with ( followed by a keyword
    # .*?         -> Match anything (non-greedy)
    # \)          -> Ends with )
    # \s*$        -> Must be at the very end of the string
    legal_pattern = re.compile(
        rf'\(\s*{legal_keywords}.*?\)\s*$', 
        re.DOTALL | re.IGNORECASE
    )
    
    legal_source = None
    match = legal_pattern.search(answer_text)
    
    if match:
        legal_source = match.group(0).strip()
        # Remove the legal source from the answer text
        answer_text = answer_text[:match.start()].strip()
        
        # Post-cleanup: Sometimes removing the source leaves a trailing "." or whitespace
        if answer_text.endswith('.'):
            answer_text = answer_text[:-1].strip()

    return {
        "id": q_id,
        "metadata": { "section": topic },
        "question": question_text,
        "answer": answer_text,
        "legal_source": legal_source
    }

def validate_count(data):
    # Extract all IDs found
    found_ids = [int(item['id']) for item in data]
    found_set = set(found_ids)
    
    expected_count = 706
    missing = [i for i in range(1, expected_count + 1) if i not in found_set]
    duplicates = [x for i, x in enumerate(found_ids) if found_ids.count(x) > 1]
    
    print(f"--- VALIDATION REPORT ---")
    print(f"Total Extracted: {len(found_ids)}")
    
    if len(found_ids) == expected_count and not missing:
        print(f"✅ SUCCESS: Found exactly {expected_count} unique questions.")
        return True
    else:
        print(f"❌ ERROR: Count mismatch.")
        if missing:
            print(f"   Missing IDs: {missing[:10]}... (Total {len(missing)})")
        if duplicates:
            print(f"   Duplicate IDs: {set(duplicates)}")
        return False

# --- USAGE ---
if __name__ == "__main__":
    file_path = "perguntas_respostas_2023_IRPF.md" # Make sure this matches your file name
    
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        data = extract_and_validate_rfb_questions(text)
        
        # Validate
        if validate_count(data):
            # Save to JSON if valid
            output_file = "perguntas_respostas_2023_IRPF.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            print(f"Saved extracted data to {output_file}")
            
    except FileNotFoundError:
        print(f"Error: Could not find file '{file_path}'")

--- VALIDATION REPORT ---
Total Extracted: 706
✅ SUCCESS: Found exactly 706 unique questions.
Saved extracted data to perguntas_respostas_2023_IRPF.json
